### **``Exploration Notebook``** 
``Equity Impact on Employee Attrition in the Workplace``

``Created by: Mijail Q. Mariano``

``13AUGUST2022``

----

In [ ]:
# notebook dependencies
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 200

# diasbling warning messages
import warnings
warnings.filterwarnings("ignore")

# importing key libraries
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

# numpy import
import numpy as np
import random

# importing acquire module
import acquire

# importing data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set(style = "darkgrid")

# plotly/visual import
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# file cleaning modules
from skimpy import clean_columns

# stats/math modules
import scipy.stats as stats
from math import sqrt

# yellowbrick recursive feature elimination-cross validation method (used for plotting accuracy)
from yellowbrick.model_selection import rfecv

# sklearn data science library
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.inspection import permutation_importance

# modules used in modeling
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, RFECV, f_regression

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import PowerTransformer

# reporting
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

# xgboost import
from xgboost import XGBClassifier

----
#### **``Initial Planning/Ideas``**

Individual Data Science Project:

Mijail Mariano

August 13th 2022

**<u>``1. Formulating the question``</u>**

``This question should be:``

* About social equity or of similar importance (i.e., inequality, racial discrimination, social-mobility, equal opportunity)
* The question is to be freamed in a way that can be quantitatively measured in terms of organizational value & also raises the question around -  “How equal/diverse or fair is an organization's current workplace?”

**target variable: "Attrition"**

**<u>``2. Exploration questions``</u>**

**``What are you attempting to predict/help to address:``**

``Employee/Company Attrition Rate``

* What is company attrition?
* Why is company attrition important?
* What are the employee attrition demographics?
* Are there pros to attrition? If so, what are these?

**``What specifically are you attempting to investigate/understand:``**

``Equity in the workplace and its impact on attrition``

*Ok, but what specifically?...*

``Do socioeconomic/location factors such as:``

* Where an employee is from/grows-up (County level) impact whether or not they remain with a company?
* The high-school graduation rate
* Incarceration/prison rate
* Fraction of population married by 35 years old
* Poverty rate
* Teenage birth rate

``Are there other questions that may be important to answer?``

How much does an employee's geographical background (where they are from) impact their decision to remain or leave the company?
Are there socioeconomic/employee demographic differences between those employees who leave the company and those who remain? (descriptive/summary statistics)

**<u>``3. Methodology``</u>**

**``Note:``** 

For this project I am assuming the company's geographical location to be New York City, NY and that employees are only from counties within the three (3) tri-state areas. This includes counties solely from the state's of Connecticut, New Jersey, and New York. To conduct the analysis I will also use a random generator to blindly assign birthplace/locations where employees grew-up and the socioeconomic variables from those locations to statistically explore these variables.

``Where’s the data from?``

To conduct this analysis and potentially generate a predictive company attrition model I combine real socioeconomic and economic data from Harvard’s Opportunity Atlast with an artificially created 2017 IBM Human Resources Kaggle dataset of a small-medium sized company (~1500 records). .

The Opportunity Atlas is a collaborative social equality project through Harvard University, the US Census Bureau, and the US Internal Revenue Service. The initiative’s aim is to track and plot socioeconomic data by exact US states, counties, cities, and neighborhoods in order to understand the childrens’ outcomes and prospect of social mobility. 

*The Atlas is composed of ~21mil Americans born between 1978-1983 who are in their mid-late thirties today. The platform and estimates are based on:

* The 2000 and 2010 Decennial Census short form
* Federal income tax returns for 1989, 1994, 1995, and 1998-2015
* Data from the American Community Survey

<u>Reference Links:</u>
* https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset
* https://www.opportunityatlas.org/

``Why couldn’t you use a real dataset?``

Given the sensitive nature of real employee information, it is relatively difficult to attain similar publicly available data from businesses. Additionally, since it is not common for organizations to collect similar socioeconomic information/drivers that I attempt to investigate - the combination of synthetic and real data seemed like an adequate method for scientific testing.

``So how should I think about this data?``

You can think about this data and the subsequent estimates as a way to understand how geographical/environmental characteristics potentially play a role in employee tenure. Additionally, these estimates may also help organizations to understand potential employee equity differences in order to address them and successfully retain essential employees. 

``Why might these employees decide to leave their company?`` 

(said another way)....
How might these demographic differences contribute to an employee’s decision to stay or leave their company?

Ok, so what happens if employers don’t retain these employees?

**<u>``4. What can employers do to retain these employees?``</u>**

(placeholder for recommendations)


``Opportunity Atlas (Equity DF): features/variables``

1. High_School_Graduation_Rate_rP_gP_pall\
Fraction of children who grew up in this area with a high school degree or a GED. Estimates have a margin of error; for example, standard error at county level for children with parents at 25th percentile is 1% pooling race and gender groups and 3% for black men. This outcome is available only at the county (not tract) level due to small sample sizes. (Source: American Community Survey)

2. Household_Income_at_Age_35_rP_gP_pall\
Average annual household income in 2014-15 for children (now in their mid-30s) who grew up in this area. Estimates have a margin of error; for example, standard error at tract level for children with parents at 25th percentile is $1,917 pooling race and gender groups and $2,721 for black men. (Source: Federal income tax records)

3. Incarceration_Rate_rP_gP_pall\
Fraction of children who grew up in this area who were in prison or jail on April 1, 2010. Estimates have a margin of error; for example, standard error at tract level for children with parents at 25th percentile is 1% pooling race and gender groups and 4% for black men. (Source: 2010 Decennial Census)

4. Fraction_Married_at_Age_35_rP_gP_pall\
Fraction of children who grew up in this area who are married in 2015 (in their mid-30s). Estimates have a margin of error; for example, standard error at tract level for children with parents at 25th percentile is 3% pooling race and gender groups and 4% for black men. (Source: Income Tax Records)

5. Poverty_Rate_in_2012-16\
Fraction of all residents of this area with household incomes below the federal poverty line in 2012-16. (Source: American Community Survey.)

6. Teenage_Birth_Rate_women_only_rP_gF_pall\
Fraction of women who grew up in this area who claimed ever a child who was born when the women were between the ages of 13 and 19 as a dependent when filing taxes. Estimates have a margin of error; for example, standard error at tract level for children with parents at 25th percentile is 4% pooling race groups and 6% for black women. (Source: Income Tax Records)


``IBM Dataset: features/variables``
1. Age
2. Attrition
3. BusinessTravel
4. DailyRate
5. Department
6. DistanceFromHome
7. Education
8. EducationField
9. EmployeeCount
10. EmployeeNumber
11. EnvironmentSatisfaction
12. Gender
13. HourlyRate
14. JobInvolvement
15. JobLevel
16. JobRole
17. JobSatisfaction
18. MaritalStatus
19. MonthlyIncome
20. MonthlyRate
21. NumCompaniesWorked
22. Over18
23. OverTime
24. PercentSalaryHike
25. PerformanceRating
26. RelationshipSatisfaction
27. StandardHours
28. StockOptionLevel
29. TotalWorkingYears
30. TrainingTimesLastYear
31. WorkLifeBalance
32. YearsAtCompany
33. YearsInCurrentRole
34. YearsSinceLastPromotion
35. YearsWithCurrManager


----

### **``Data Acquisition and Preparation``**

In [ ]:
# let's import the IBM employee data first

ibm_df = pd.read_csv("/Users/mijailmariano/Desktop/IBM_HR-Employee-Attrition.csv")
print()
print(f'IBM dataset shape: {ibm_df.shape}')
ibm_df.head()

In [ ]:
# let's import the opportunity atlas data

equity_df = pd.read_csv("/Users/mijailmariano/Desktop/equity_table.csv")
print()
print(f'Equity dataset shape: {equity_df.shape}')
equity_df.head()

In [ ]:
# replacing/removing the word "miles" in distance

# equity_df["distance"] = equity_df["distance"].str.replace("miles", "").astype(int)
# equity_df.dtypes.sort_values()

In [ ]:
# number of unique county distances

# equity_df["distance"].nunique()

In [ ]:
# let's use pandas' qcut method to parse out distance groups

# intervals = pd.Series(pd.cut(
#         equity_df["distance"], 
#         bins = equity_df["distance"].nunique(), 
#         duplicates = "drop").sort_values().tolist())

# intervals.unique()

In [ ]:
# viewing counties by distance sorted

# equity_df[["county_name", "distance"]].sort_values("distance")

In [ ]:
# setting counties by distance
# [(0.971, 3.9], (3.9, 6.8], (6.8, 9.7], (9.7, 12.6], (18.4, 21.3], (24.2, 27.1], (27.1, 30.0]]

# area_one = equity_df[equity_df["distance"] <= 5].county_name.tolist()
# area_two = equity_df[(equity_df["distance"] > 5) & (equity_df["distance"] <= 10)].county_name.tolist()
# area_three = equity_df[(equity_df["distance"] > 10) & (equity_df["distance"] <= 21)].county_name.tolist()
# area_four = equity_df[(equity_df["distance"] > 21) & (equity_df["distance"] <= 27)].county_name.tolist()
# area_five = equity_df[(equity_df["distance"] > 27) & (equity_df["distance"] <= 30)].county_name.tolist()

# print(area_one)
# print('----------------------------------------------------')
# print(area_two)
# print('----------------------------------------------------')
# print(area_three)
# print('----------------------------------------------------')
# print(area_four)
# print('----------------------------------------------------')
# print(area_five)

In [ ]:
# creating a function to randomly apply county based on the employee's distance from home

def get_county(x, lst_a, lst_b, lst_c, lst_d, lst_e):
        '''where x = employees' work distance from home in miles. 
        function will iterate through all records and randomly assign a county based on distance from work.'''
        lst = []

        if x <= 5:
                county = random.choice(lst_a)
                lst.append(county)

        elif x > 5 and x <= 10:
                county = random.choice(lst_b)
                lst.append(county)

        elif x > 10 and x <= 21:
                county = random.choice(lst_c)
                lst.append(county)
        
        elif x > 27 and x <= 30:
                county = random.choice(lst_e)
                lst.append(county)

        else:
                county = lst_d[0]
                lst.append(county)

        # returning the list of counties
        return lst

In [ ]:
# creating a random generated county list 

# random.seed(548)
# county_lst = ibm_df["DistanceFromHome"].apply(get_county, args = (area_one, area_two, area_three, area_four, area_five))
# # let's flatten the county list
# county_lst = [val for sublist in county_lst for val in sublist]
# county_lst[:5]

In [ ]:
# let's create a pandas series and assign the county list to the ibm dataframe

# county_lst = pd.Series(county_lst)
# county_lst.shape

In [ ]:
# assigning the county series to the ibm dataframe

# ibm_df["county_name"] = county_lst
# ibm_df.head() # checks out!

In [ ]:
# merging the two tables/dfs on county name
# bringing over socioeconomic data from equity_df 

# emp_df = ibm_df.merge(
#     equity_df,
#     how = "left",
#     left_on = "county_name",
#     right_on = "county_name"
# ).drop(columns = "distance")

# emp_df.head() # checks out!

In [ ]:
# catching the dataframe

# emp_df.to_csv("/Users/mijailmariano/codeup-data-science/drivers_of_workplace_equity/emp_df.csv", index = False)

In [ ]:
# importing the dataframe 

df = acquire.get_employee_df()
df.head()

In [ ]:
# can consider removing/dropping the following features/columns
# "over_18": all employees meet this criteria
# "employee_count": redundant information

df = acquire.clean_employee_df(df)
df.head()

In [ ]:
# let's check the df info

df.info()

In [ ]:
# initial summary statistics

summary_stats = df.describe().T
summary_stats["range"] = summary_stats["max"] - summary_stats["min"]
summary_stats.sort_index()

In [ ]:
# let's loop through and inspect columns and unique values

for col in df.columns:
    print(f'Column: {col.upper()}')
    print(f'Date type: {df[col].dtype}')
    print(f'Missing values: {df[col].isnull().any()}')
    print(f'Number of unique values: {df[col].nunique()}')
    print(f'Data Sample: {list(df[col].head(10).sort_values())}')
    print('-------------------------------------------------------------------')

In [ ]:
# let's examine the target variable

att_mean = df["attrition"].mean()

plt.figure(figsize = (4, 4))
sns.set(font_scale = 0.5)

ax = sns.countplot(
    x = "attrition",
    data = df,
    order = df["attrition"].value_counts().index,
    palette = "crest_r")

ax.bar_label(ax.containers[0])

plt.title(f'Company Attrition Rate: {att_mean:.2%}')
plt.xlabel(None)
plt.ylabel("Employees")
plt.show

In [ ]:
# plotting individual columns/features by data type

for col in df.columns:
    if df[col].dtype == int or df[col].dtype == float:
        plt.figure(figsize = (7, 2))
        sns.histplot(
            df[col],
            color = "seagreen",
            alpha = 0.4,
            kde = True)

        plt.title(f'Feature: {col}')
        plt.xlabel(None)
        plt.show()
    
    elif col == "cty" or col == "county_name": 
        # treating large discrete count plots seperate
        plt.figure(figsize = (7, 2))
        sns.countplot(
            df[col],
            order = df[col].value_counts().index,
            label = col, 
            palette="crest_r")

        plt.legend()
        # plt.xticks(rotation = 90)
        plt.tick_params(
                        axis='x', # changes apply to the x-axis
                        rotation = 90,
                        labelsize = 4)
        plt.xlabel(None)
        plt.title(f'Feature: {col}')
        plt.show()

    else:
        plt.figure(figsize = (7, 2))
        sns.countplot(
            y = df[col],
            order = df[col].value_counts().index, 
            orient = "h", 
            palette="crest_r")

        plt.ylabel(None)
        plt.title(f'Feature: {col}')
        plt.show()

In [ ]:
# classifiying features by data type = discrete/categorical or continuous 

disc_lst = df.select_dtypes(exclude = "number").columns.sort_values().tolist()
cont_lst = df.select_dtypes(include = "number").columns.sort_values().tolist()

print(f'discrete variables:\n{disc_lst}')
print()
print(f'continuous variables:\n{cont_lst}')

In [ ]:
# box plots for continuous variables: are there outliers in the features?

for col in cont_lst:
    plt.figure(figsize = (7, 2))

    sns.boxplot(
        df[col],
        orient = "h", 
        palette = "crest", 
        width= 0.3)

    plt.title(f'variable: {col}')
    plt.ylabel(None)
    plt.xlabel(None)
    plt.show()

In [ ]:
# continuous variables: lower and upper bounds using interquartile (IQR) range
'''Function created to determine continuous variable/feature lower/upper bounds using an interquartile range method'''
def get_lower_and_upper_bounds(df):
    holder = []
    num_lst = df.select_dtypes("number").columns.tolist()
    # num_lst = [ele for ele in num_lst if ele not in ("parcel_id", 'longitude', 'latitude', 'blockgroup_assignment')]
    k = 1.5

    # determining continuous features/columns
    for col in df[num_lst]:
        
        # determing 1st and 3rd quartile
        q1, q3 = df[col].quantile([.25, 0.75])
        
        # calculate interquartile range
        iqr = q3 - q1
        
        # set feature/data lower bound limit
        lower_bound = q1 - k * iqr

        # set feature/data upperbound limit
        upper_bound = q3 + k * iqr
        
        metrics = { 
            "column": col,
            "column type": df[col].dtype,
            "iqr": round(iqr, 5),
            "lower_bound": round(lower_bound, 5),
            "lower_outliers": len(df[df[col] < lower_bound]),
            "upper_bound": round(upper_bound, 5),
            "upper_outliers": len(df[df[col] > upper_bound])
        }

        holder.append(metrics)

    new_df = pd.DataFrame(holder)

    # returning the cleaned dataset
    print(f'dataframe shape: {new_df.shape}')

    return new_df

In [ ]:
# applying the function to return a lower/upperbounds dataframe

get_lower_and_upper_bounds(df).sort_values(by="upper_outliers", ascending=False).reset_index(drop = True)

#### ``Outlier Observations``

**Intentionally choosing to leave the socioeconomic/opportunity atlas data for testing. Additionally, since these are actual community figures and may subsequently impact an employees attrition decision.**

<u>Ater applying an interquartile range with a k value of 1.5, I will apply/ommit the following outlier cleaning:</u>

**monthly_income:** upperbound $16581.00

- this signals to me that these employees are either contractors or senior leaders at the company. If they are senior leaders, my assumption is that they are more reluctant to make an employment change due to their potential leadership responsibilities, role, and stated salary. 

**year_since_last_promotion:** upperbound 7.50 years

- this signals to me that these employees may already be seeking other opportunities elsewhere - potentially with an employer that is willing to offer them a better position with greater responsibilities, and salary. I will ommit employees over the dataset upperbound.

**years_at_company:** upperbound 18.00 years

- this signals to me that these employees may be reluctant to leave their current employer given the length of their tenure. Additionally, the current median employee tenure in the US is ~5 years, I will ommit employees over this upperbound.

**total_working_years:** upperbound 28.50 year

- similar to above, these employees may also be enroute to retirement or are simply less willing to make a career/employment shift given this late into their careers. Additionally, they might have also learned ways in which they can successfully navigate the socioeconomic challenges apparent in their communities. I will ommit employees over this upperbound.

**years_in_current_role:** upperbound 14.50 years

- given the median US employee/employer tenure, I assume that these employees 1. may already be seeking other opportunities or 2. are comfortable in their current roles and thefore are less willing to take socioeconomic factors as reason to leave their company. 

**years_with_curr_manager:** upperbound 14.50 years

- same as above, yet there could also be a relationship factor thats prevelant in the employees' tenure. Employees may have a good working relationship with their managers where they potentially feel "heard" and are having their socioeconomic concerns addressed. I will ommit employees over this upperbound.

----

``references:``

- https://www.bls.gov/news.release/tenure.nr0.htm


In [ ]:
# creating a function to clean outliers at upperbounds

def df_outliers(df):

    # monthly income / leadership or seniority
    df = df[df["monthly_income"] <= 16581.00]
    
    # length of working tenure
    df = df[df["total_working_years"] <= 28.00]

    # length of tenure at current company
    df = df[df["years_at_company"] <= 18.00]

    # number of years since last promotion
    df = df[df["years_since_last_promotion"] <= 7.50]

    # number of years in current role 
    df = df[df["years_in_current_role"] <= 14.50]

    # number of year with current manager
    df = df[df["years_with_curr_manager"] <= 14.50]

    # returning the cleaned dataset
    print(f'dataframe shape: {df.shape}')

    return df

In [ ]:
# applying the outlier function

df = df_outliers(df)
df.head()

In [ ]:
# percentage lost after outlier cleaning
# ~19.5% loss of original df

round((df.shape[0] - ibm_df.shape[0])/df.shape[0], 3)

In [ ]:
# seeing the dataset

df.head()

In [ ]:
# renaming/classifying attrition as boolean T/F values

df["attrition"] = df["attrition"].replace({"Yes": True, "No": False})
df.attrition.head()

----
### **``Splitting the Original Dataset``**


In [ ]:
'''Function created to split the initial dataset into train, validate, and test datsets'''
def train_validate_test_split(df):
    train_and_validate, test = train_test_split(
                                                df, 
                                                test_size = 0.2, 
                                                random_state = 548,
                                                stratify = df["attrition"])
    
    train, validate = train_test_split(
                                    train_and_validate,
                                    test_size = 0.3,
                                    random_state = 548,
                                    stratify = train_and_validate["attrition"])

    print(f'train shape: {train.shape}')
    print(f'validate shape: {validate.shape}')
    print(f'test shape: {test.shape}')

    return train, validate, test

In [ ]:
# splitting dataset into training, validate, and test datasets

train, validate, test = train_validate_test_split(df)

In [ ]:
# what is the percentage of the target variable in ea. dataset?
# this checks out!

print(f'target percentage in train: {round(len(train[train["attrition"] == True])/train.shape[0], 3)}')
print(f'target percentage in validate: {round(len(validate[validate["attrition"] == True])/validate.shape[0], 3)}')
print(f'target percentage in test: {round(len(test[test["attrition"] == True])/test.shape[0], 3)}')

----
#### ``Setting a Baseline: Prediction Employee Attrition``

In [ ]:
# let's set an attrition baseline using a mode method for ea. dataset
# For the baseline accuracy, I have taken the mode of the two (2) binary "attrition" options = False and have set this as my random prediction
# baseline accuracy score: the total number of times that the baseline prediction matched the actual employee attrition outcome
# baseline accuracy score: ~82.0% **(note that if the goal is to predict attrition = 1, then baseline accuracy is ~18%)
    
train_baseline = train
train_baseline["baseline_prediction"] = False
baseline_train = (train_baseline["baseline_prediction"] == train_baseline["attrition"]).mean().round(3)
print(f'Training Baseline Accuracy: % {(baseline_train * 100).round(5)}')

print('-------------------------------------------')

validate_baseline = validate
validate_baseline["baseline_prediction"] = False
baseline_val = (validate_baseline["baseline_prediction"] == validate_baseline["attrition"]).mean().round(3)
print(f'Validate Baseline Accuracy: % {(baseline_val * 100).round(5)}')

In [ ]:
# current continuous variables in dataset

train.select_dtypes(include = "number").columns.tolist()

In [ ]:
# let's also look at discrete variables

train.select_dtypes(exclude = "number").columns.tolist()

In [ ]:
# setting continuous variables list

cont_lst = sorted([
'employee_age',
'monthly_income',
'percent_salary_hike',
'total_working_years',
'training_times_last_year',
'years_at_company',
'household_income_at_35',
'high_school_graduation_rate',
'percentage_married_by_35',
'incarceration_rate',
'women_teenage_birthrate',
'poverty_rate',
'employment_rates_at_35yrs',
'single_parent_frac',
'years_since_last_promotion',
'years_in_current_role',
'years_with_curr_manager',
])

cont_lst

In [ ]:
# setting disc variables list

disc_lst = sorted([
'stock_option_level',
'work_life_balance',
'education',
'job_involvement',
'job_level',
'job_satisfaction',
'performance_rating',
'relationship_satisfaction',
'county_name',
'state',
'department',
'education_field',
'gender',
'job_role',
'marital_status',
'environment_satisfaction'
])

disc_lst

-----
### ``Exploration: Hypothesis Testing``

    The focus will first be on determined equity/community variables

**"Attrition":** Discrete/Categorical Target Variable

**``Variables not taken into exploration:``**

1. 'business_travel'
2. 'cty'
3. 'daily_rate'
4. 'distance_from_home'
5. 'hourly_rate'
6. 'monthly_rate'
7. 'num_companies_worked'
8. 'over_time'


**``Continuous Variables:``**

1. 'employee_age'
2. 'employment_rates_at_35yrs'
3. 'high_school_graduation_rate'
4. 'household_income_at_35'
5. 'incarceration_rate'
6. 'monthly_income'
7. 'percent_salary_hike'
8. 'percentage_married_by_35'
9. 'poverty_rate'
10. 'single_parent_frac'
11. 'total_working_years'
12. 'training_times_last_year'
13. 'women_teenage_birthrate'
14. 'years_at_company'
15. 'years_since_last_promotion'
16. 'years_in_current_role'
17. 'years_with_curr_manager'

**``Discrete Varibles:``**

1. 'attrition'
2. 'county_name'
3. 'department'
4. 'education'
5. 'education_field'
6. 'environment_satisfaction'
7. 'gender'
8. 'job_involvement'
9. 'job_level'
10. 'job_role'
11. 'job_satisfaction'
12. 'marital_status'
13. 'performance_rating'
14. 'relationship_satisfaction'
15. 'standard_hours'
16. 'state'
17. 'stock_option_level'
18. 'work_life_balance'


In [ ]:
# updating datasets with confirmed variables

train = train[[
            'attrition',
            'employee_age',
            'monthly_income',
            'percent_salary_hike',
            'total_working_years',
            'training_times_last_year',
            'years_at_company',
            'household_income_at_35',
            'high_school_graduation_rate',
            'percentage_married_by_35',
            'incarceration_rate',
            'women_teenage_birthrate',
            'poverty_rate',
            'employment_rates_at_35yrs',
            'single_parent_frac',
            'years_since_last_promotion',
            'county_name',
            'department',
            'education',
            'education_field',
            'environment_satisfaction',
            'gender',
            'job_involvement',
            'job_level',
            'job_role',
            'job_satisfaction',
            'marital_status',
            'performance_rating',
            'relationship_satisfaction',
            'state',
            'stock_option_level',
            'work_life_balance',
            'years_in_current_role',
            'years_with_curr_manager'
]]

validate = validate[[ 
            'attrition',
            'employee_age',
            'monthly_income',
            'percent_salary_hike',
            'total_working_years',
            'training_times_last_year',
            'years_at_company',
            'household_income_at_35',
            'high_school_graduation_rate',
            'percentage_married_by_35',
            'incarceration_rate',
            'women_teenage_birthrate',
            'poverty_rate',
            'employment_rates_at_35yrs',
            'single_parent_frac',
            'years_since_last_promotion',
            'county_name',
            'department',
            'education',
            'education_field',
            'environment_satisfaction',
            'gender',
            'job_involvement',
            'job_level',
            'job_role',
            'job_satisfaction',
            'marital_status',
            'performance_rating',
            'relationship_satisfaction',
            'state',
            'stock_option_level',
            'work_life_balance',
            'years_in_current_role',
            'years_with_curr_manager'
]]

test = test[[ 
            'attrition',
            'employee_age',
            'monthly_income',
            'percent_salary_hike',
            'total_working_years',
            'training_times_last_year',
            'years_at_company',
            'household_income_at_35',
            'high_school_graduation_rate',
            'percentage_married_by_35',
            'incarceration_rate',
            'women_teenage_birthrate',
            'poverty_rate',
            'employment_rates_at_35yrs',
            'single_parent_frac',
            'years_since_last_promotion',
            'county_name',
            'department',
            'education',
            'education_field',
            'environment_satisfaction',
            'gender',
            'job_involvement',
            'job_level',
            'job_role',
            'job_satisfaction',
            'marital_status',
            'performance_rating',
            'relationship_satisfaction',
            'state',
            'stock_option_level',
            'work_life_balance',
            'years_in_current_role',
            'years_with_curr_manager'
]]

In [ ]:
# cleaning dataframes for needed variables

# train[disc_lst] = train[disc_lst].astype(object)
# validate[disc_lst] = validate[disc_lst].astype(object)
# test[disc_lst] = test[disc_lst].astype(object)

# train.info()

In [ ]:
# 2nd split: splitting larger datasets into x and y variables

X_train = train.drop(columns = "attrition")
y_train = train['attrition']

X_validate = validate.drop(columns = "attrition")
y_validate = validate['attrition']

X_test = test.drop(columns = "attrition")
y_test = test['attrition']

In [ ]:
# checking the target variable
# note: the model will also read boolean type (F/T) as either (0/1)
# False (did not churn):    566 employees
# True (did churn):     122 employees

y_train.value_counts()

----
### ``Hypothesis Tests: Continuous Variables``

$H_0$: The variable mean of those who leave the company is not statistically different than the population mean.

$H_a$: The variable mean of those who leave the company is statistically different than the population variable mean.

$\alpha$ = 1 - confidence level (95% confidence level)

$\alpha$: 0.05

In [ ]:
# sns.pairplot (first 6)

sns.pairplot(
    train, 
    vars =[
        'employee_age',
        'employment_rates_at_35yrs',
        'high_school_graduation_rate',
        'household_income_at_35',
        'incarceration_rate'],
    corner = True,  
    hue = "attrition",
    height = 2,
    markers=["o", "D"])

plt.show()

In [ ]:
# plotting continuous variables against target

for col in cont_lst:
    plt.figure(figsize=(6, 2))

    sns.scatterplot(
        train[col],
        train["years_at_company"],
        hue = train["attrition"],
        s = 4)

    plt.legend(
        bbox_to_anchor=(1.105, 1), 
        title = "attrition", 
        loc = 'upper right', 
        borderaxespad = 0)

    plt.title(f'{col} and company_tenure')
    plt.xlabel(None)
    plt.show()

In [ ]:
# continuous variables and evaluating statistical signifance in sample mean vs. population mean
import scipy.stats as stats 

alpha = 0.05

metrics = []
for col in cont_lst:
    pop_mean = train[col].mean()
    # sample_2_mean = train[train["attrition"] == False][col]
    sample_1_mean = train[train["attrition"] == True][col]

    t_score, p_value = stats.ttest_1samp(sample_1_mean, pop_mean)

    if p_value < alpha:
        output = {
            "continuous_feature": col,
            "t_score": t_score,
            "p_value": p_value}
        
        metrics.append(output)

    else:
        print(f'Column: {col} not statistically significant.')
        print("--------------------------------------")

onesample_t_test_scores = pd.DataFrame(metrics)
onesample_t_test_scores.round(4)

In [ ]:
onesample_t_test_scores.continuous_feature.tolist()

In [ ]:
train.head()

In [ ]:
# sns.heatmap

# taking needed variables/df sample
corr_df = train[[ 
    'employee_age',
    'employment_rates_at_35yrs',
    'high_school_graduation_rate',
    'household_income_at_35',
    'monthly_income',
    'percentage_married_by_35',
    'poverty_rate',
    'total_working_years',
    'women_teenage_birthrate',
    'years_at_company',
    'years_in_current_role',
    'years_with_curr_manager']].reset_index(drop = True)

# returning correlation coefficient 
corr_array = corr_df.corr()

# create the object and axes
fig, ax = plt.subplots(figsize=(10, 8))
sns.set(style = "white")

# mask
mask = np.triu(np.ones_like(corr_array, dtype=np.bool))

# adjust mask and df
mask = mask[1:, :-1]
corr = corr_array.iloc[1:,:-1].copy()

g = sns.heatmap(
    corr, 
    mask=mask,
    cmap = "Blues",
    vmin = -1, 
    vmax = 1, 
    annot = True,
    annot_kws={
        'fontsize': 12
    },
    fmt =".2f",
    linewidths = 0.2,
    square = True)

g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.show()

----

#### **``Summary: 1-sample T-test Results``**

After initial continuous hypotheses testing against the population mean - we can conclude that the following features/variables hold a statistical relationship with the target:

1. age
2. employment rates at 35yrs
3. high school graduation rate
4. household income at 35
5. monthly income
6. percentage married by 35
7. poverty rate
8. total working years
9. women teenage birthrate
10. years at company
11. years in current role
12. years with curr manager

<u>questions/thoughts after analysis:</u>

there are several features/variables that I will want to further investigate against the population. For example, I wonder if certain features such as "the percentage of single parents" in communities, or the "total number of years since an employee's last promotion" may be less linear and tests such as non-parametric/linear relationship testing may be more suitable for evaluating them against a population.

----
### **``Hypothesis Tests: Categorical/Discrete Variables``**

$H_0$: "There is NO association/relationship between observed variable outcomes and expected employee attrition."

$H_a$: "There IS an association/relationship between observed variable outcomes and expected employee attrition."

$\alpha$ = 1 - confidence level (95% confidence level)

$\alpha$: 0.05

In [ ]:
# sns.swarmplot

for col in disc_lst:
    plt.figure(figsize=(12, 3))
    sns.set(font_scale = .7)
    sns.swarmplot(x = col, y = "years_at_company", data = train, hue = "attrition", palette = "BuPu")

    plt.xlabel(None)
    plt.ylabel("Company Tenure")
    plt.xticks(fontsize = 6, rotation = 45)
    plt.title(col)

    plt.legend(
        bbox_to_anchor=(1.1, 1), 
        title = "attrition",
        loc = "upper right")
    plt.show()

In [ ]:
# looping through discrete variables and conducting Chi-Squared test on target variable

alpha = 0.05
metrics = []

for col in disc_lst:
    # generating the ChiSquared Test and returning results
    observed = pd.crosstab(index = train[col], columns = train["attrition"], margins = True)

    chi, p_value, degf, exp_values = stats.chi2_contingency(observed)

    if p_value < alpha:
        output = {
            "discrete_feature": col,
            "chi2": chi,
            "degs_of_freedom": degf,
            "p_value": p_value}
        
        metrics.append(output)

    else:
        print(f'variable: {col}')
        print('Not statistically significant.')
        print('---------------------------------------')

chi2_results = pd.DataFrame(metrics)
chi2_results.round(5)

----

#### **``Summary: Chi_Squared Results``**

After initial discrete/categorical hypotheses testing against the "attrition" target variable - we can conclude that the following features/variables hold a statistical relationship with the target:

- job level
- job role
- marital status
- stop option level

<u>questions/thoughts after analysis:</u>

features such as job level and job role on its surface may not appear to be similar, however I presume that there may be some relationship amongst "titles"/"roles" and the level. For instance, certain job levels may only be applicable to a particular role such as a Director or Senior Vice President. 

I observe that there are only 4 distinct job levels and 9 distinct job roles at this company. On a second comb through of the data, I will want to conduct summary statistics and statistical testing across these features to investigate if there is any relationship amongst the independent variables.


----

### **``Feature Selection & Scaling``**

In [ ]:
# selecting statistically significant variables/features from testing

X_train = X_train[[
'job_level', 
'job_role', 
'marital_status', 
'stock_option_level',
'employee_age',
'employment_rates_at_35yrs',
'high_school_graduation_rate',
'household_income_at_35',
'monthly_income',
'percentage_married_by_35',
'poverty_rate',
'total_working_years',
'women_teenage_birthrate',
'years_at_company',
'years_in_current_role',
'years_with_curr_manager']]

X_validate = X_validate[[
'job_level', 
'job_role', 
'marital_status', 
'stock_option_level',
'employee_age',
'employment_rates_at_35yrs',
'high_school_graduation_rate',
'household_income_at_35',
'monthly_income',
'percentage_married_by_35',
'poverty_rate',
'total_working_years',
'women_teenage_birthrate',
'years_at_company',
'years_in_current_role',
'years_with_curr_manager']]

X_test = X_test[[
'job_level', 
'job_role', 
'marital_status', 
'stock_option_level',
'employee_age',
'employment_rates_at_35yrs',
'high_school_graduation_rate',
'household_income_at_35',
'monthly_income',
'percentage_married_by_35',
'poverty_rate',
'total_working_years',
'women_teenage_birthrate',
'years_at_company',
'years_in_current_role',
'years_with_curr_manager']]

X_train.shape

----
#### **``Standard Scaler``**

In [ ]:
# i will fit and evaluate Sklearn's Standard Scaler and a Robust Scaler

cont_lst = X_train.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()

for col in cont_lst:
    plt.figure(figsize=(10, 3))
    sns.set(font_scale = .8)

    scaler = StandardScaler()
    scaler.fit(X_train[[col]])
    x_scaled = scaler.transform(X_train[[col]])


    plt.subplot(121)
    ax1 = sns.histplot(X_train[[col]], bins = 25, edgecolor = 'black', label = 'original data')

    # removing axes scientific notation 
    ax1.ticklabel_format(style = "plain") 
    plt.title(f'Original: {col}')
    plt.legend()

    plt.subplot(122)
    ax2 = sns.histplot(x_scaled, bins=25, edgecolor = 'black', label = "scaled data")

    # removing axes scientific notation 
    ax2.ticklabel_format(style = "plain") 
    plt.title(f'Scaled: {col}')
    plt.legend()

----
#### **``Robust Scaler``**

In [ ]:
# Robust Scaler

cont_lst = X_train.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()


for col in cont_lst:
    plt.figure(figsize=(10, 3))
    sns.set(font_scale = .8)

    scaler = RobustScaler()
    scaler.fit(X_train[[col]])
    x_scaled = scaler.transform(X_train[[col]])


    plt.subplot(121)
    ax1 = sns.histplot(X_train[[col]], bins = 25, edgecolor = 'black', label = 'original data')

    # removing axes scientific notation 
    ax1.ticklabel_format(style = "plain") 
    plt.title(f'Original: {col}')
    plt.legend()

    plt.subplot(122)
    ax2 = sns.histplot(x_scaled, bins=25, edgecolor = 'black', label = "scaled data")

    # removing axes scientific notation 
    ax2.ticklabel_format(style = "plain") 
    plt.title(f'Scaled: {col}')
    plt.legend()

----
#### **``Scaler Summary:``**

- ``Since outlier management was previously handled in preparation and I don't see this factor being prevelant in the graphs, I will choose to use sklearn's Standard Scaler to transform the selected continuous variables.``

In [ ]:
# scaling continuous features/data with sklearn "StandardScaler"
# creating function to do this

def scaled_data(df, scaled_cols):
    # creating a copy of the original zillow/dataframe
    df_scaled = df.copy()

    scaler = StandardScaler()

    scaler.fit(df_scaled[scaled_cols])

    df_scaled[scaled_cols] = scaler.transform(df_scaled[scaled_cols])

    # returning newly created dataframe with scaled data
    return df_scaled

In [ ]:
# selecting features to scale

scale_lst = X_train.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()
scale_lst

In [ ]:
# viewing the head for comparison
X_train.head()

In [ ]:
# generating new X_train, and X_validate datasets w/scaled data

X_train = scaled_data(X_train, scale_lst)
X_validate = scaled_data(X_validate, scale_lst)

# preview the data
X_train.head() # checks out!

----
#### **``Assigning and Creating Dummy Variables for Discrete Features``**


In [ ]:
X_train.select_dtypes(exclude = "number").columns.tolist()

In [ ]:
'''function to create dummy variables for discrete variables/feature'''
def get_dummy_dataframes(train_df, val_df, test_df):

    # train dataset
    train_dummy = pd.get_dummies(
        data = train_df, 
        columns = [
                'job_level', 
                'job_role', 
                'marital_status', 
                'stock_option_level'],
        drop_first = False, 
        dtype = bool)

    # validate dataset
    validate_dummy = pd.get_dummies(
        data = val_df, 
        columns = [
                'job_level', 
                'job_role', 
                'marital_status', 
                'stock_option_level'],
        drop_first = False, 
        dtype = bool)

    # test dataset
    test_dummy = pd.get_dummies(
        data = test_df, 
        columns = [
                'job_level', 
                'job_role', 
                'marital_status', 
                'stock_option_level'],
        drop_first = False, 
        dtype = bool)

    # returning dummy datasets
    return train_dummy, validate_dummy, test_dummy

In [ ]:
# inspecting df before dummy transformation

X_train.shape

In [ ]:
# transforming dataframe to include dummy variables

train_dummy, validate_dummy, test_dummy = get_dummy_dataframes(X_train, X_validate, X_test)

# cleaning column names after dummy transformation
train_dummy = clean_columns(train_dummy)
validate_dummy = clean_columns(validate_dummy)
test_dummy = clean_columns(test_dummy)

print(f'shape: {train_dummy.shape}')
train_dummy.head()

----
### **``Feature Selection``**

In [ ]:
# creating a recursive feature function to select and predict the top performing features in input the datasets

def recursive_feature_eliminate(X_train, y_train, number_of_top_features):

    # initialize the ML algorithm
    lr = LogisticRegression()

    rfe = RFE(lr, n_features_to_select = number_of_top_features)

    # fit the data using RFE
    rfe.fit(X_train, y_train) 

    # get the mask of the columns selected
    feature_mask = rfe.support_

    # get list of the column names
    rfe_features = X_train.iloc[:,feature_mask].columns.tolist()

    # view list of columns and their ranking
    # get the ranks using "rfe.ranking" method
    variable_ranks = rfe.ranking_

    # get the variable names
    variable_names = X_train.columns.tolist()

    # combine ranks and names into a df for clean viewing
    rfe_ranks_df = pd.DataFrame({'Feature': variable_names, 'Ranking': variable_ranks})

    # sort the df by rank
    return rfe_ranks_df.sort_values('Ranking')


In [ ]:
# SKLearn's Recursive Feature Elimination function

recursive_feature_eliminate(train_dummy, y_train, 10).reset_index(drop = True)

In [ ]:
# using sklearn's RFECV function to select best features to include w/Random Forest classifier

cross_validation = RFECV(
    estimator = RandomForestClassifier(random_state = 548),
    min_features_to_select = 5)

cross_validation = cross_validation.fit(train_dummy, y_train)

rf_features = train_dummy.columns[cross_validation.support_].tolist()
pd.DataFrame(rf_features).rename(columns = {0: "Features"}).sort_values("Features").reset_index(drop = True)

In [ ]:
# applying RFECV w/Logistic Regression estimator

cross_validation = RFECV(
    estimator = LogisticRegression(
        C = 1, 
        class_weight = "balanced", 
        random_state = 548),
        min_features_to_select = 5)

cross_validation = cross_validation.fit(train_dummy, y_train)

lr_features = train_dummy.columns[cross_validation.support_].tolist()
pd.DataFrame(lr_features).rename(columns = {0: "Features"}).sort_values("Features").reset_index(drop = True)

----
#### **``Summary: Recursive Feature Elimination``**

I will leverage Sklearn's RFECV using a Random Forest estimator to select features used in Modeling. 

In [ ]:
# let's apply the RFECV Feature using a Random Forest Classifier to model the datasets

train_model = train_dummy[rf_features]
validate_model = validate_dummy[rf_features]
test_model = test_dummy[rf_features]

train_model.head()

In [ ]:
# looping through Random Forest depth & sample leaf

leaf_counter = 0

for i in range(1, 11):
    
    # Make the model
    rf = RandomForestClassifier(max_depth=i, min_samples_leaf = (leaf_counter + 1), random_state=548)
    leaf_counter += 1

    # Fit the model (on train dataset)
    rf = rf.fit(train_model, y_train)

    # We'll evaluate the model's performance on train, first
    y_predictions = rf.predict(train_model)

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_train, y_predictions, output_dict=True)
    print(f"Random Forest with max_depth of: {i}")
    print(f"Random Forest with minimum sample leaves of: {leaf_counter}")
    print(pd.DataFrame(report))

    print()

In [ ]:
# looping through Decision Tree Classifier and working backwards from number of leaves

leaf_counter = 0

for i in range(10, 0, -1):

    # Make the model
    tree = DecisionTreeClassifier(max_depth=i, min_samples_leaf = (leaf_counter + 1), random_state=548)
    leaf_counter += 1

    # Fit the model (on train dataset)
    tree = tree.fit(train_model, y_train)

    # We'll evaluate the model's petreeormance on train, first
    y_predictions = tree.predict(train_model)

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_train, y_predictions, output_dict=True)
    print(f"Decision Tree with max_depth of: {i}")
    print(f"Decision Tree with minimum sample leaves of: {leaf_counter}")
    print(pd.DataFrame(report))
    
    print()

In [ ]:
# evaluating the Decision Tree model with
# max depth: 8
# number of sample leaves: 3

# creating the Decision Tree Model
tree1 = DecisionTreeClassifier(
    max_depth = 8, 
    min_samples_leaf = 3,
    random_state = 548)

# fitting the Decision Tree model
tree1 = tree1.fit(train_model, y_train)

decision_tree_acc = tree1.score(train_model, y_train)

print('Accuracy of Decision Tree Classifer on training dataset: {:.2f}'.format(decision_tree_acc))
print(f'Absolute % Difference (Baseline vs. Decision Tree Model): % {round(0.82 - decision_tree_acc, 2)*100}')

In [ ]:
# making prediction on train dataset and returning confusion matrix against actual observations

y_predictions = tree1.predict(train_model)

pd.crosstab(y_predictions, y_train).rename_axis( "Predicted", axis = 0)

In [ ]:
# plotting recursive feature elimination - cross validations & F1 score using yellowbrick rfecv

# plt.figure(figsize=(12,4))

# cv = StratifiedKFold(5)
# visualizer = rfecv(
#     RandomForestClassifier(random_state = 548), 
#     X = train_model, 
#     y = y_train, 
#     cv = cv, 
#     scoring='f1_weighted')

In [ ]:
# calculating and plotting feature importance using Decision tree1 Classifer / non-recursive elimination

plt.figure(figsize = (10, 5))
sns.set(style = "darkgrid", font_scale = .75)

# organizing/ordering feature importance by value
sorted_idx = tree1.feature_importances_.argsort()

sns.barplot(tree1.feature_importances_[sorted_idx], train_model.columns[sorted_idx], orient = "h", color = "b")

plt.title("Feature Importance")
plt.show()

In [ ]:
# showing decision tree importance by feature

pd.DataFrame(
    tree1.feature_importances_, 
    index = train_model.columns).\
    rename(columns = {0: "feature_importance"}).\
    sort_index(ascending=True).\
    sort_values(by = 'feature_importance', ascending = False)

In [ ]:
'''Feature for plotting decision tree classifier - feature importance'''
def plot_feature_importance(X_train, y_train, depth, leaves):
    plt.figure(figsize = (10, 5))
    sns.set(style = "darkgrid", font_scale = .75)

    tree = DecisionTreeClassifier(
        max_depth = depth,
        min_samples_leaf = leaves,
        random_state = 548)
    tree = tree.fit(X_train, y_train)

    sorted_idx = tree.feature_importances_.argsort()

    sns.barplot(
        tree.feature_importances_[sorted_idx], 
        X_train.columns[sorted_idx], 
        orient = "h", 
        palette = "crest")

    plt.title("Feature Importance")

    plt.show()

In [ ]:
# let's also plot feature importance for just opportunity atlas features

equity_lst = [col for col in train_dummy.columns if 'rate' in col or 'household' in col or 'by' in col]
equity_df = train_dummy[equity_lst]

equity_df.head()
plot_feature_importance(equity_df, y_train, 4, 2)

In [ ]:
# Atlas Decision Tree w/max_depth = 4, and min_sample_leaf = 2

plt.figure(figsize=(12, 8))

unscaled_atlas = X_train[equity_lst]

tree_atlas = DecisionTreeClassifier(max_depth=4, min_samples_leaf=2, random_state=548)
tree_atlas = tree_atlas.fit(unscaled_atlas, y_train)

plot_tree(tree_atlas, feature_names = unscaled_atlas.columns.astype("str"), class_names = y_train.unique().astype("str"))

plt.title("Opportunity Atlas: Decision Tree")
plt.show()

----
### **``Modeling``**

<u>Generating the Following Models:</u>

1. XGboost
2. Polynomial/Logistic Regression 
3. Decision Tree
4. Random Forest
5. Naive Bayes
6. KNN

In [ ]:
# model num 1: xgboost

metrics = []

# creating the model
boost = XGBClassifier()

# fitting the model (on train and only train)
boost = boost.fit(train_model, y_train)

# applying the model and evaluating its performance on the training dataset
in_sample_accuracy = boost.score(train_model, y_train)

# next, we'll evaluate the model on "out-of-sample" data (validate)
out_of_sample_accuracy = boost.score(validate_model, y_validate)

output = {
    "model": "XGboost", \
    "train_accuracy": in_sample_accuracy, \
    "validate_accuracy": out_of_sample_accuracy
}

metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)


In [ ]:
# model num 2A: Logistic Regression

metrics = []

for i in np.linspace(0.1, 1.0, 10):

    # creating the model
    logi = LogisticRegression(
        C = i, 
        random_state=548)

    # fitting the model (on train and only train)
    logi = logi.fit(train_model, y_train)

    # applying the model and evaluating its performance on the training dataset
    in_sample_accuracy = logi.score(train_model, y_train)

    # next, we'll evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = logi.score(validate_model, y_validate)

    output = {
        "model": "logi_model", \
        "C_parameter": i, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }

    metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)

In [ ]:
# creating an opportunity atlas df from validate 

val_equity = validate_dummy[equity_lst]
val_equity.head()

In [ ]:
# model num 2B: Logistic Regression / Opportunity Atlas Data

metrics = []

for i in np.linspace(0.1, 1.0, 10):

    # creating the model
    logi2 = LogisticRegression(
        C = i,
        random_state=548)

    # fitting the model (on train and only train)
    logi2 = logi2.fit(equity_df, y_train)

    # applying the model and evaluating its performance on the training dataset
    in_sample_accuracy = logi2.score(equity_df, y_train)

    # next, we'll evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = logi2.score(val_equity, y_validate)

    output = {
        "model": "logi2_model", \
        "C_parameter": i, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }

    metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)

In [ ]:
# model num 3A: Decision Tree

metrics = []
leaf_counter = 0

for i in range(1, 21):
    # creating the model
    tree = DecisionTreeClassifier(
        min_samples_leaf = (leaf_counter + 1), 
        max_depth = i,
        random_state=548)

    # increasing leaf counter by 1
    leaf_counter += 1

    # fitting the model
    tree = tree.fit(train_model, y_train)

    # applying the model and evaluating its performance on the training dataset
    in_sample_accuracy = tree.score(train_model, y_train)

    # next, we'll evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = tree.score(validate_model, y_validate)

    output = {
        "model": "decision_tree", \
        "max_depth": i, \
        "min_sample_leaves": leaf_counter, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }

    metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)

In [ ]:
# model num 3B: Decision Tree (reversed sample leaves)

metrics = []
leaf_counter = 21

for i in range(1, 21):
    # creating the model
    tree = DecisionTreeClassifier(
        min_samples_leaf = (leaf_counter - 1), 
        max_depth = i, 
        random_state=548)

    # decreasing leaf counter by 1
    leaf_counter -= 1

    # fitting the model
    tree = tree.fit(train_model, y_train)

    # applying the model and evaluating its performance on the training dataset
    in_sample_accuracy = tree.score(train_model, y_train)

    # next, we'll evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = tree.score(validate_model, y_validate)

    output = {
        "model": "decision_tree_reversed", \
        "max_depth": i, \
        "min_sample_leaves": leaf_counter, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }

    metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)

In [ ]:
# model num 3C: Decision Tree (Opportunity Atlas / reversed sample leaves)

metrics = []
leaf_counter = 0

for i in range(1, 21):
    # creating the model
    tree = DecisionTreeClassifier(
        min_samples_leaf = (leaf_counter + 1), 
        max_depth = i, 
        random_state=548)

    # decreasing leaf counter by 1
    leaf_counter += 1

    # fitting the model
    tree = tree.fit(equity_df, y_train)

    # applying the model and evaluating its performance on the training dataset
    in_sample_accuracy = tree.score(equity_df, y_train)

    # next, we'll evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = tree.score(val_equity, y_validate)

    output = {
        "model": "decision_tree_atlas", \
        "max_depth": i, \
        "min_sample_leaves": leaf_counter, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }

    metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)

In [ ]:
# model num 4A: Random Forest Tree 

metrics = []

for i in range(1, 21):
    # Make the model
    rf = RandomForestClassifier(
        min_samples_leaf = i, 
        max_depth = i,
        random_state=548)

    # Fit the model (on train and only train)
    rf = rf.fit(train_model, y_train)

    # Use the model
    # We'll evaluate the model's performance on train, first
    in_sample_accuracy = rf.score(train_model, y_train)
    
    # next, we'll evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = rf.score(validate_model, y_validate)
    
    output = {
        "model": "rf_classifier",
        "max_depth": i, \
        "num_of_sample_leaf": i, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.train_accuracy)
df.round(2)


In [ ]:
# plotting the Random Forest 

# plotting the "Random Forest" comparison across in-sample and out-sample datasets:

sns.set_theme("paper")

i_range = range(1, 21)
train_scores = []
validate_scores = []

for i in i_range:
    rf = RandomForestClassifier(
        min_samples_leaf = i, 
        max_depth = i,
        random_state=548)
        
    rf.fit(train_model, y_train)

    train_scores.append(rf.score(train_model, y_train))
    validate_scores.append(rf.score(validate_model, y_validate))


plt.figure(figsize=(12, 4))
plt.xlabel('Max Depth & Sample Leaf')
plt.ylabel('accuracy')
plt.plot(i_range, train_scores, label='Train')
plt.plot(i_range, validate_scores, label='Validate')
plt.legend()
plt.xticks(i_range)
plt.show()

In [ ]:
# model num 4B: Random Forest Tree Reverse Count

metrics = []
leaf_count = 21

for i in range(1, 21):
    # Make the model
    rf = RandomForestClassifier(
        min_samples_leaf = (leaf_count-1), 
        max_depth = i,
        random_state=548)
        
    leaf_count -= 1

    # Fit the model (on train and only train)
    rf = rf.fit(train_model, y_train)

    # Use the model
    in_sample_accuracy = rf.score(train_model, y_train)
    
    # next, evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = rf.score(validate_model, y_validate)
    
    output = {
        "model": "rf_reversed",
        "max_depth": i, \
        "num_of_sample_leaf": leaf_count, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.train_accuracy)
df.round(4)

In [ ]:
# plotting the "Random Forest - reverse count" comparison across in-sample and out-sample datasets:
# consider plotting secondary X-axis to represent sample leaf count

sns.set_theme("paper")

i_range = range(1, 21)
train_scores = []
validate_scores = []
leaf_counter = 21

for i in i_range:
    rf = RandomForestClassifier(
        min_samples_leaf = (leaf_counter - 1), 
        max_depth = i,
        random_state=548)
        
    leaf_counter -= 1
    rf.fit(train_model, y_train)
    train_scores.append(rf.score(train_model, y_train))
    validate_scores.append(rf.score(validate_model, y_validate))

plt.figure(figsize=(12, 4))
plt.xlabel('Max Depth')
plt.ylabel('accuracy')
plt.plot(i_range, train_scores, label='Train')
plt.plot(i_range, validate_scores, label='Validate')
plt.legend()
plt.xticks(i_range)
plt.show()

In [ ]:
# model number 5A: Gaussian Bayes
# will want to return back to this model

metrics = []

# creating the model
gsb = GaussianNB()

# fitting the model
gsb = gsb.fit(train_model, y_train)

# applying the model and evaluating its performance on the training dataset
in_sample_accuracy = gsb.score(train_model, y_train)

# next, we'll evaluate the model on "out-of-sample" data (validate)
out_of_sample_accuracy = gsb.score(validate_model, y_validate)

output = {
    "model": "gaussian_bayes", \
    "train_accuracy": in_sample_accuracy, \
    "validate_accuracy": out_of_sample_accuracy
}

metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)

In [ ]:
# model number 5B: Gaussian Bayes (hypertuned)

metrics = []

for i in range(2, 15):

    # fitting the powertransformer
    pwr_trans = PowerTransformer()
    pwr_trans = pwr_trans.fit(train_model)

    # transforming training/validate data
    Data_Transformed_1 = pwr_trans.transform(train_model)
    Data_Transformed_2 = pwr_trans.transform(validate_model)

    params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

    cv_method = RepeatedStratifiedKFold(
        n_splits = i,  
        n_repeats = 3, 
        random_state = 548)

    # creating the Gaussian model
    gsb = GaussianNB()

    # creating the hypertuned-model
    gs_NB = GridSearchCV(
                    estimator = gsb, 
                     param_grid = params_NB, 
                     cv = cv_method,
                     verbose = 1 , 
                     scoring = 'accuracy')

    # fitting the model
    gs_NB = gs_NB.fit(Data_Transformed_1, y_train)

    # applying the model and evaluating its performance on the training dataset
    in_sample_accuracy = gs_NB.score(Data_Transformed_1, y_train)

    # next, we'll evaluate the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = gs_NB.score(Data_Transformed_2, y_validate)

    output = {
        "model": "gaussian_bayes", \
        "K-folds": i, \
        "train_accuracy": in_sample_accuracy, \
        "validate_accuracy": out_of_sample_accuracy
    }

    metrics.append(output)

df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.validate_accuracy)
df.round(4)

In [ ]:
# plotting the "KNN" comparison across in-sample and out-sample datasets:

sns.set_theme("paper")

n_range = range(1, 21)
train_scores = []
validate_scores = []

for n in n_range:
    knn = KNeighborsClassifier(
        n_neighbors = n, 
        weights = 'uniform')

    knn.fit(train_model, y_train)
    train_scores.append(knn.score(train_model, y_train))
    validate_scores.append(knn.score(validate_model, y_validate))

plt.figure(figsize=(12, 4))
plt.xlabel('Number of Neighbors/observations')
plt.ylabel('accuracy')
plt.plot(n_range, train_scores, label='Train')
plt.plot(n_range, validate_scores, label='Validate')
plt.legend()
plt.xticks([0,5,10,15,20])
plt.show()

In [ ]:
# model number 6A: K-nearest neighbor (KNN)

metrics = []

for k in range(1, 21):
    # Make the model
    knn = KNeighborsClassifier(
                                n_neighbors = k, 
                                weights = 'uniform')

    # Fit the model (on train and only train)
    knn = knn.fit(train_model, y_train)

    # evaluating the model's performance on training dataset
    in_sample_accuracy = knn.score(train_model, y_train)
    
    # evaluating the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = knn.score(validate_model, y_validate)

    output = {
        "num_of_neighbors": k,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy}
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.train_accuracy)
df.round(4)

In [ ]:
# model number 6B: K-nearest neighbor (KNN) / Opportunity Atlas

metrics = []

for k in range(1, 21):
    # Make the model
    knn = KNeighborsClassifier(
                                n_neighbors = k, 
                                weights = 'uniform')

    # Fit the model (on train and only train)
    knn = knn.fit(equity_df, y_train)

    # evaluating the model's performance on training dataset
    in_sample_accuracy = knn.score(equity_df, y_train)
    
    # evaluating the model on "out-of-sample" data (validate)
    out_of_sample_accuracy = knn.score(val_equity, y_validate)

    output = {
        "num_of_neighbors": k,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy}
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["percent_change_diff"] = ((df.train_accuracy - df.validate_accuracy) / df.train_accuracy)
df.round(4)

----
### **``Summary Notes: Modeling``**

After comparing all six (6) unique models, I conclude that the following models resulted in the best predictive overall accuracy performance:

**Logistic Regression (1): C = 1.0**

    - ~84% predictive accuracy
    - relative training set diff. ~0%

**Logistic Regression (2): C = 0.9**

    - ~84% predictive accuracy
    - relative training set diff. ~%0

**Decision Tree (1): depth of 13, and min sample leaf of 13**

    - ~84% predictive accuracy
    - relative training set diff. ~1% (+)

**Decision Tree (2): depth of 7, and min sample leaf of 14**

    - ~84% predictive accuracy
    - relative training set diff. ~1% (+)

**Random Forest: depth of 7, and min sample leaf of 14**

    - ~84% predictive accuracy
    - relative training set diff. ~1% (+)

**K-nearest Neighbor (KNN): k = 4**

    - ~84% predictive accuracy
    - relative training set diff. ~1% (+)

**K-nearest Neighbor (KNN): k = 10**

    - ~84% predictive accuracy
    - relative training set diff. ~1% (+)

-----
#### **``Summary: Model Evaluation``**

After evaluating all the models tested, I am choosing to deploy the Logistic Regression model for several reasons:

- ease of understanding/interpretation
- may be most flexible to deploy/maintain over time
    - compared to commonly collected information which is key to algorithms such as decision tree/random forests

In [ ]:
# deploying the Logistic Regression w/C of 1.0 on Test Dataset

# creating the model
logi_final = LogisticRegression(
            C = 1, 
            random_state=548)

# fitting the model (on train dataset)
logi_final = logi_final.fit(train_model, y_train)

# applying the model and evaluating its performance on all three (3) datasets

train_accuracy = logi_final.score(train_model, y_train)
validate_accuracy = logi_final.score(validate_model, y_validate)
test_accuracy = logi_final.score(test_model, y_test)

In [ ]:
# returning final results including relative difference test dataset 

train_diff = (baseline_train - train_accuracy)/train_accuracy
val_diff = (train_accuracy - validate_accuracy)/validate_accuracy
test_diff = (validate_accuracy - test_accuracy)/test_accuracy

pd.DataFrame({'dataset': ["baseline", "train", "validate", "test (final)"], "accuracy": [baseline_train, train_accuracy, validate_accuracy, test_accuracy], "relative_difference": [0, train_diff, val_diff, test_diff]})

In [ ]:
# Logistic Regression Report on Validate

metrics = []

for i in np.linspace(0.1, 1.0, 10):

    # creating the model
    logi = LogisticRegression(
        C = i, 
        random_state=548)

    # fitting the model (on train and only train)
    logi = logi.fit(train_model, y_train)

    # We'll evaluate the model's petreeormance on train, first
    y_predictions = logi.predict(validate_model)

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_validate, y_predictions, output_dict=True)
    print(f"With Class Weight: {i.round(4)}")
    print(pd.DataFrame(report))
    
    print()

In [ ]:
# returning final accuracy report

# generating predictions for test dataset w/Logistic Regression model
y_predictions = logi_final.predict(test_model)

# Produce the classification report on the actual y values and this model's predicted y values
report = classification_report(y_test, y_predictions, output_dict = True)

final_report = pd.DataFrame(report).round(4)
final_report

In [ ]:
# confusion matrix on test dataset
# model unssessfully predicted any of the 43 turn-over employees in the test dataset
# however, the model accurately predicted all instances when employee did not leave the company

pd.crosstab(y_predictions, y_test).rename_axis( "Predicted", axis = 0)

----
### **``Generating Predictions for Remaining Employees``**

In [ ]:
# let's deploy the model on the full ibm dataset and measure its performance

df = acquire.get_employee_df()

df.head()

In [ ]:
# cleaning dataframe for current employees only

df = df[df["Attrition"] == "No"]

print(f'shape after cleaning: {df.shape}')
df.head()

In [ ]:
# initial cleaning 

df = acquire.clean_employee_df(df)

df.head() # checks out!

In [ ]:
# treating dataset for outliers

df = acquire.df_outliers(df)

df.head() # checks out!

In [ ]:
# creating an unscaled df copy for add predictions 

df_original = df.copy()
df_original.head() # checks out!

In [ ]:
# transform the dataset

scale_lst = df.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()
df = acquire.scaled_data(df)

print(f'shape: {df.shape}')
df.head()

In [ ]:
# create dummy variables

df_dummy = pd.get_dummies(
data = df, 
columns = [
        'job_level', 
        'job_role', 
        'marital_status', 
        'stock_option_level'],
drop_first = False, 
dtype = bool)

df_dummy.head()

In [ ]:
# clean the dummy dataframe

df_cleaned = clean_columns(df_dummy)

print(df_cleaned.shape)
df_cleaned.head()

In [ ]:
# created needed predictions dataframe

df_model = df_cleaned[rf_features]

print(df_model.shape)
df_model.head()

In [ ]:
# generating probabilities 

y_probabilities = logi_final.predict_proba(df_model)
y_probabilities[0:20] # checks out!

In [ ]:
# converting the probabilities into a Pandas Dataframe will make it easier to manipulate

probabilities = pd.DataFrame(y_probabilities).rename(columns = {0: "attrition_remains", 1: "attrition_turns_over"})

print(probabilities.shape)
probabilities

In [ ]:
# generating predictions for the ibm w/Logistic Regression model

predictions = logi_final.predict(df_model)
y_predictions = pd.DataFrame(predictions).rename(columns = {0: "predicted_turn_over"})

print(y_predictions.shape)
print(f'unique values: {y_predictions.nunique()}')
y_predictions.head() # checks out!

In [ ]:
# getting employee numbers

employee_numbers = pd.DataFrame(df_original.index).rename(columns = {0: "employee_number"})
employee_numbers.head() # checks out!

In [ ]:
# let's tie it all together

print(type(probabilities), probabilities.shape)
print(type(y_predictions), y_predictions.shape)
print(type(employee_numbers), employee_numbers.shape)

In [ ]:
# final dataframe

final_df = pd.concat([employee_numbers, probabilities, y_predictions], axis = 1)

In [ ]:
# printing the results

print(f'final df shape: {final_df.shape}')
final_df.head()

In [ ]:
# predicted turn over: 35 employees or ~3.5% of current employees

final_df["predicted_turn_over"].value_counts()

In [ ]:
# generating a predictions csv

final_df.to_csv("/Users/mijailmariano/codeup-data-science/drivers_of_workplace_equity/emp_predictions.csv", index = False)

----
### **<u>``Appendix: Measuring Model Performance on Overall Data``</u>**

In [ ]:
# import df

df = acquire.get_employee_df()
df.head()

In [ ]:
# initial cleaning

df = acquire.clean_employee_df(df)
df.head()

In [ ]:
# clean for outliers

df = acquire.df_outliers(df)
df.head()

In [ ]:
# scale the data

scale_lst = df.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()
df = acquire.scaled_data(df)

print(df.shape)
df.head()

In [ ]:
# generate dummy variables

df_dummy = pd.get_dummies(
                        data = df, 
                        columns = [
                                'job_level', 
                                'job_role', 
                                'marital_status', 
                                'stock_option_level'],
                        drop_first = False, 
                        dtype = bool)

df_dummy.head()

In [ ]:
# clean the dummy dataframe

df_cleaned = clean_columns(df_dummy)

print(df_cleaned.shape)
df_cleaned.head()

In [ ]:
# generate model df for predictions

df_model = df_cleaned[rf_features]

print(df_model.shape)
df_model.head()

In [ ]:
# generate predictions

y_predictions = logi_final.predict(df_model)
y_predictions[:20]

In [ ]:
# generating a confusion matrix

pd.crosstab(index = y_predictions, columns = df_cleaned["attrition"], margins=True).rename_axis("predicted", axis = 0)

In [ ]:
# understanding the confusion matrix further:

TN, FP, FN, TP = confusion_matrix(df_cleaned["attrition"], y_predictions).ravel()

print(f'True Negative: {TN}')
print(f'False Positive: {FP}')
print(f'False Negative: {FN}')
print(f'True Positive: {TP}')

In [ ]:


ALL = TP + FP + FN + TN

accuracy = (TP + TN)/ALL
true_positive_rate = sensitivity = recall = power = TP/(TP+FN)
precision = PPV = TP/(TP+FP)
f1_score = 2*(precision*recall)/(precision+recall)

false_positive_rate = false_alarm_ratio = fallout = FP/(FP+TN)
true_negative_rate = specificity = selectivity = TN/(TN+FP)
false_negative_rate = miss_rate = FN/(FN+TP)

support_pos = TP + FN
support_neg = FP + TN

indx_lst = [
        "Accuracy", 
        "True Positive Rate/Sensitivity/Recall/Power", 
        "False Positive Rate/False Alarm Ratio/Fall-out", 
        "True Negative Rate/Specificity/Selectivity", 
        "False Negative Rate/Miss Rate", 
        "Precision/PPV", 
        "F1 Score", 
        "Support (False)", 
        "Support (True)"
        ]

metric = [
        accuracy.round(4),
        true_positive_rate.round(4),
        false_positive_rate.round(4),
        true_negative_rate.round(4),
        false_negative_rate.round(4),
        precision.round(4),
        f1_score.round(4),
        support_neg.round(4),
        support_pos.round(4)
]

performance_report = pd.DataFrame(metric, indx_lst).rename(columns = {0: "Performance"})

In [ ]:
# returning performance report

performance_report